Searching and Downloading Data with Sunpy
=========================================

In this notebook we will look at how sunpy can be used to search and download solar physics data. Here is what this notebook contains:

1. Overview of Fido
2. Attributes of Fido
3. Constructing a data search query
4. Inspecting data search query
5. More complex queries
6. Using Fido to query the HEK
7. Downloading data
8. Extending Fido : New clients and use for your own packages <img src="https://raw.githubusercontent.com/sunpy/sunpy.org/main/_static/img/sunpy_icon.svg" alt="logo" width="100" align="right"/>


Fido Unified Downloader
====================

## 1. Overview of Fido



* Fido is sunpy's interface for searching and downloading solar physics data.


* It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.


* Offers a way to search and accesses multiple instruments and all available data providers in a single query.


* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido offers access to data available through:

 * **VSO**
 * **JSOC** (~~through `drms`~~)
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **HEK**
 * **HELIO**
 
Lets first import `Fido` and the `attrs` submodule, and inspect Fido. 
As described here Fido provides access to many sources of data through different `clients`, these clients can be defined inside sunpy or in other packages. Lets print the current list of available clients within sunpy.

In [1]:
from sunpy.net import Fido, attrs as a

In [4]:
Fido

Client,Description
CDAWEBClient,Provides access to query and download from the Coordinated Data Analysis Web (CDAWeb).
EVEClient,Provides access to Level 0CS Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to the GOES XRS fits files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
...,...
NoRHClient,Provides access to the Nobeyama RadioHeliograph (NoRH) averaged correlation time series data.
RHESSIClient,Provides access to the RHESSI observing summary time series data.
HEKClient,Provides access to the Heliophysics Event Knowledgebase (HEK).
HECClient,Provides access to the HELIO webservices.


## 2. Using attributes to search for data with Fido

Sunpy uses specified *attributes* to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail.

First we can look at a.Time, which is used to specify the timerange of a query.

In [5]:
a.Time("2021-02-12 18:00", "2021-02-12 23:00")

<sunpy.net.attrs.Time(2021-02-12 18:00:00.000, 2021-02-12 23:00:00.000)>

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.

In [6]:
a.Instrument

Attribute Name,Client,Full Name,Description
aia,VSO,AIA,Atmospheric Imaging Assembly
bcs,VSO,BCS,Bragg Crystal Spectrometer
be_continuum,VSO,BE-Continuum,INAF-OACT Barra Equatoriale Continuum Instrument
be_halpha,VSO,BE-Halpha,INAF-OACT Barra Equatoriale Hα Instrument
bigbear,VSO,Big Bear,"Big Bear Solar Observatory, California TON and GONG+ sites"
caii,VSO,CAII,Kanzelhöhe Ca II k Instrument
cds,VSO,CDS,Coronal Diagnostic Spectrometer
celias,VSO,CELIAS,"Charge, Element, and Isotope Analysis System"
cerrotololo,VSO,Cerro Tololo,"Cerro Tololo, Chile GONG+ site"
chp,VSO,chp,Chromospheric Helium-I Imaging Photometer


Sunpy also now provides tab completion to auto-fill the attribute name

In [7]:
a.Instrument.wispr

<sunpy.net.attrs.Instrument(WISPR: Wide-Field Imager for Solar Probe Plus) object at 0x7fdd8a23ddc0>

To search for certain wavelengths, we need to specify the input as an `astropy Quantity` which is a the combination of a value and an associated unit. This is something is universal in the sunpy stack - that every physical input/output is a `Quantity`.

In [8]:
from astropy import units as u

In [9]:
171 * u.angstrom

<Quantity 171. Angstrom>

In [10]:
a.Wavelength(17.1*u.nm)

<sunpy.net.attrs.Wavelength(170.99999999999997, 170.99999999999997, 'Angstrom')>

## 3. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [11]:
result = Fido.search(a.Time("2021-02-12 18:00:00", "2021-02-12 23:00:00"), 
                     a.Instrument("AIA"))#,a.Wavelength(171*u.angstrom))

The `Fido.search` will return `UnifiedResponse` object with all the information available from the search that fit the criteria specified by the attrs. This `result` can then be passed to `Fido.fetch` to download the queried data (We will discuss this further below). Lets now inspect our `result` which holds the results of our query, if we print this we should see a summary of our query:

In [12]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Extra Flags,Info
,,,,Angstrom,,,,,,,Mibyte,,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str1,str106
2021-02-12 18:00:00.000,2021-02-12 18:09:13.000,SDO,AIA,335.0 .. 335.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:04.000,2021-02-12 18:09:17.000,SDO,AIA,193.0 .. 193.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.000 to 2.000 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:05.000,2021-02-12 18:00:06.000,SDO,AIA,4500.0 .. 4500.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,S,"AIA level 1, 4096x4096 [0.300 exposure] [100.00 percentd]"
2021-02-12 18:00:05.000,2021-02-12 18:09:18.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:06.000,2021-02-12 18:09:19.000,SDO,AIA,131.0 .. 131.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:09.000,2021-02-12 18:09:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,3038.47656,S,"AIA level 1, 4096x4096 [47 records] [0 eclipse] [0 darks] [2.000 to 2.000 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:09.000,2021-02-12 18:09:10.000,SDO,AIA,211.0 .. 211.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,2973.82812,S,"AIA level 1, 4096x4096 [46 records] [0 eclipse] [0 darks] [2.901 to 2.901 exposure] [100.00 avg. percentd]"
2021-02-12 18:00:11.000,2021-02-12 18:09:12.000,SDO,AIA,94.0 .. 94.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,2973.82812,S,"AIA level 1, 4096x4096 [46 records] [0 eclipse] [0 darks] [2.902 to 2.902 exposure] [100.00 avg. percentd]"


Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `Wavelength` attribute is passed as an astropy Quantity:

In [13]:
result = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom))

In [14]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:00:21.000,2021-02-12 18:00:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:00:33.000,2021-02-12 18:00:34.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:00:45.000,2021-02-12 18:00:46.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:00:57.000,2021-02-12 18:00:58.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:01:09.000,2021-02-12 18:01:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:01:21.000,2021-02-12 18:01:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:01:33.000,2021-02-12 18:01:34.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


We can further specify this query by choosing the cadence (time-sampling) of the data we want to search for and download. This can be achieved by using the `Sample` attribute. Similar to the `Wavelength` attributes, this needs to be an `astropy Quantity`. Lets further specify the search above to only search for data with a cadence of 2 minutes.

In [15]:
result = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom),
                     a.Sample(10*u.min))

In [16]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:10:09.000,2021-02-12 18:10:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:20:09.000,2021-02-12 18:20:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:30:09.000,2021-02-12 18:30:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:40:09.000,2021-02-12 18:40:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:50:09.000,2021-02-12 18:50:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 19:00:09.000,2021-02-12 19:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 19:10:09.000,2021-02-12 19:10:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


## 4. Dealing with search results and the `UnifiedResponse`

So far we have shown how queries can be constructed and returned as a `UnifiedResponse` object. This object can then be passed to `fido.fetch` to download the data. However lets first inspect the `UnifiedResponse` object to get a better idea of what it consists of. 

From above, we can see that what is returned from a `Fido.search` is the `UnifiedResponse` object that consists of a one record (for a simple query) or multiple records depending on the search. These records (stored as tables) can be accessed by indexing the `UnifiedResponse` object. It acts both like a two dimensional array, where the first dimension is the response index and the second index is the row index, and a dictionary where you can index the responses by the name of the client. 


In this case here, we only have one query returned as it was a simple query for AIA data

In [17]:
result.file_num

30

As mentioned above the first index is the response index, and the second is the row, so we can pull out the individual rows. In this example, there is only one response (a simple query)

So we can index to get the first row of the query results

In [18]:
result[0, 0]

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info,fileid
,,,,Angstrom,,,,,,,Mibyte,,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64,str24
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]",aia__lev1:171:1392228047


The normal slices operations that work with other Python sequences will also work here (i.e. result[0, ::5] will return every fifth row etc

In [20]:
result[0,::10]

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 19:40:09.000,2021-02-12 19:40:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 21:20:09.000,2021-02-12 21:20:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


### Inspecting the result tables

Each of these tables indexed from the `UnifiedResponse` object are `QueryResponseTable` objects which are just `astropy.table` objects that can easily interacted with (e.g. indexed, filtered etc).

This is particularly handy when the results that are searched for are from metadata tables such as the HEK! But lets first look at how we can interact with the results table, and filter etc. Lets first look at how we can play around with each table

In [21]:
result[0].colnames

['Start Time',
 'End Time',
 'Source',
 'Instrument',
 'Wavelength',
 'Provider',
 'Physobs',
 'Wavetype',
 'Extent Width',
 'Extent Length',
 'Extent Type',
 'Size',
 'Info',
 'fileid']

In [22]:
result[0]["Start Time"]

<Time object: scale='utc' format='iso' value=['2021-02-12 18:00:09.000' '2021-02-12 18:10:09.000'
 '2021-02-12 18:20:09.000' '2021-02-12 18:30:09.000'
 '2021-02-12 18:40:09.000' '2021-02-12 18:50:09.000'
 '2021-02-12 19:00:09.000' '2021-02-12 19:10:09.000'
 '2021-02-12 19:20:09.000' '2021-02-12 19:30:09.000'
 '2021-02-12 19:40:09.000' '2021-02-12 19:50:09.000'
 '2021-02-12 20:00:09.000' '2021-02-12 20:10:09.000'
 '2021-02-12 20:20:09.000' '2021-02-12 20:30:09.000'
 '2021-02-12 20:40:09.000' '2021-02-12 20:50:09.000'
 '2021-02-12 21:00:06.000' '2021-02-12 21:10:09.000'
 '2021-02-12 21:20:09.000' '2021-02-12 21:30:09.000'
 '2021-02-12 21:40:09.000' '2021-02-12 21:50:09.000'
 '2021-02-12 22:00:09.000' '2021-02-12 22:10:09.000'
 '2021-02-12 22:20:09.000' '2021-02-12 22:30:09.000'
 '2021-02-12 22:40:09.000' '2021-02-12 22:50:09.000']>

The responses can also be indexed like a dictionary (useful for more complex searches)

In [23]:
result["VSO", ::5]

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 18:50:09.000,2021-02-12 18:50:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 19:40:09.000,2021-02-12 19:40:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 20:30:09.000,2021-02-12 20:30:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 21:20:09.000,2021-02-12 21:20:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 22:10:09.000,2021-02-12 22:10:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


## 5.  More complex queries

In addition to making a query to one client for one instrument, Fido allows the flexibility to search for data from multiple instruments, wavelengths, times etc, even when the data is being obtained through different clients.

This query can be constructed by using the pipe `|` operator, which joins queries together just like the `OR` operator.

Lets now make a query that searches for both GOES/XRS and Fermi/GBM data over a particular time period

COMMENT from Sandy: Point out known warnings when they pop up.

In [25]:
result = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                     a.Instrument.xrs  | a.Instrument.gbm)

/home/jovyan/users_conda_envs/sunpy-tutorial/lib/python3.9/site-packages/astropy/table/table.py:3486: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


In [26]:
result

<sunpy.net.fido_factory.UnifiedResponse object at 0x7fdd88ef9b80>
Results from 2 Providers:

2 Results from the XRSClient:
Source: https://umbra.nascom.nasa.gov/goes/fits

       Start Time               End Time        Instrument ... Source Provider
----------------------- ----------------------- ---------- ... ------ --------
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999        XRS ...   GOES     NOAA
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999        XRS ...   GOES     NOAA

20 Results from the GBMClient:
Source: https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily

       Start Time               End Time        ... Resolution Detector
----------------------- ----------------------- ... ---------- --------
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n0
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n1
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n2
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n3
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n4
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n5
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n6
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n7
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n8
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n9
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n0
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n1
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n2
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n3
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n4
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n5
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n6
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n7
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n8
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n9

If we then want to further specify our search for each instrument, e.g. lets only search for data from the GOES 15 satellite and for detector n5 for GBM, this can be achieved by using the `&` operator 

In [27]:
result = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                     a.Instrument.xrs & a.goes.SatelliteNumber(17) | a.Instrument.gbm & a.Detector("n5"))

/home/jovyan/users_conda_envs/sunpy-tutorial/lib/python3.9/site-packages/astropy/table/table.py:3486: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


In [28]:
result

<sunpy.net.fido_factory.UnifiedResponse object at 0x7fdd893898e0>
Results from 2 Providers:

1 Results from the XRSClient:
Source: https://umbra.nascom.nasa.gov/goes/fits

       Start Time               End Time        Instrument ... Source Provider
----------------------- ----------------------- ---------- ... ------ --------
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999        XRS ...   GOES     NOAA

2 Results from the GBMClient:
Source: https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily

       Start Time               End Time        ... Resolution Detector
----------------------- ----------------------- ... ---------- --------
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      cspec       n5
2021-02-12 00:00:00.000 2021-02-12 23:59:59.999 ...      ctime       n5

### Dealing with `UnifiedResponse` query results for complex queries

In [29]:
len(result)

2

In [30]:
result[0]

Start Time,End Time,Instrument,SatelliteNumber,Physobs,Source,Provider
Time,Time,str3,int64,str10,str4,str4
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,XRS,17,irradiance,GOES,NOAA


In [31]:
result[1]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
Time,Time,str3,str4,str5,str4,str5,str2
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,GBM,flux,FERMI,NASA,ctime,n5


In [32]:
result["XRS"]

Start Time,End Time,Instrument,SatelliteNumber,Physobs,Source,Provider
Time,Time,str3,int64,str10,str4,str4
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,XRS,17,irradiance,GOES,NOAA


In [33]:
result["GBM"]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
Time,Time,str3,str4,str5,str4,str5,str2
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,GBM,flux,FERMI,NASA,ctime,n5


Similarly, as mentioned above the first index is the response index, and the second is the row, so we can pull out  the individual rows. For example, say we only want the first file from the Fermi/GBM data

In [34]:
result["GBM", 0]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector,url
Time,Time,str3,str4,str5,str4,str5,str2,str101
2021-02-12 00:00:00.000,2021-02-12 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5,https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2021/02/12/current/glg_cspec_n5_210212_v00.pha


In [35]:
result.all_colnames

['End Time',
 'Instrument',
 'Physobs',
 'Provider',
 'SatelliteNumber',
 'Source',
 'Start Time',
 'url']

In [36]:
result[1]["url"]

https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2021/02/12/current/glg_cspec_n5_210212_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2021/02/12/current/glg_ctime_n5_210212_v00.pha


## 6. Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via `parfive`), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded. 

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data

In [37]:
result = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom),
                     a.Sample(10*u.min))

In [38]:
files = Fido.fetch(result)

Files Downloaded:   0%|          | 0/30 [00:00<?, ?file/s]

These files are downloaded to a local location set in the sunpy.config.file, which by default is `~/sunpy/data/`. `Fido.fetch` returns a `parfile.Results` object which gives the path to where the files are downloaded to 

In [39]:
print(files)

['/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t18_10_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t21_20_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t19_50_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t22_50_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t18_00_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t19_00_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t20_30_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t22_10_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t18_20_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t21_30_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t20_00_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/aia_lev1_171a_2021_02_12t18_50_09_35z_image_lev1.fits', '/home/jovyan/sunpy/data/ai

You can also define what directory you want the files to be saved to by passing the directory path to the `path` keyword in Fido.fetch. For example, I want to download these files to a local directory `/Users/laurahayes/event_analysis/`

In [40]:
result[0][::10]

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
Time,Time,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str64
2021-02-12 18:00:09.000,2021-02-12 18:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 19:40:09.000,2021-02-12 19:40:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-02-12 21:20:09.000,2021-02-12 21:20:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


In [41]:
files = Fido.fetch(result[0][::10], path='./downloaded_data/')

Files Downloaded:   0%|          | 0/3 [00:00<?, ?file/s]

In [42]:
print(files)

['downloaded_data/aia_lev1_171a_2021_02_12t21_20_09_35z_image_lev1.fits', 'downloaded_data/aia_lev1_171a_2021_02_12t19_40_09_35z_image_lev1.fits', 'downloaded_data/aia_lev1_171a_2021_02_12t18_00_09_35z_image_lev1.fits']


You can also use the properties of the query to define the path to where to save the files, for example you can save them in a subdirectory named after the instrument of the queried files:

COMMENT from Jack: what properties of the search can you use to do the download file templating

In [43]:
files = Fido.fetch(result[0][::10], path='./downloaded_data/{instrument}/')

Files Downloaded:   0%|          | 0/3 [00:00<?, ?file/s]

In [44]:
print(files)

['downloaded_data/AIA/aia_lev1_171a_2021_02_12t18_00_09_35z_image_lev1.fits', 'downloaded_data/AIA/aia_lev1_171a_2021_02_12t19_40_09_35z_image_lev1.fits', 'downloaded_data/AIA/aia_lev1_171a_2021_02_12t21_20_09_35z_image_lev1.fits']


### Download errors
The parfive.Results that is returned from `Fido.fetch` also contains information on the failed downloads and the error that is associated with them. These can be accessed by:

In [47]:
files.errors

[]

However for this example, there were no errors, but in the case that there was, this would list the errors. If there were failed downloads, the ~~parfive.Results~~ can be passed back into `Fido.fetch` to retry downloading the failed downloads e.g.

COMMENT from Jack: remove mentions of `parfive` in the notebook

In [48]:
Fido.fetch(files)

Files Downloaded: 0file [00:00, ?file/s]

['downloaded_data/AIA/aia_lev1_171a_2021_02_12t19_40_09_35z_image_lev1.fits', 'downloaded_data/AIA/aia_lev1_171a_2021_02_12t21_20_09_35z_image_lev1.fits', 'downloaded_data/AIA/aia_lev1_171a_2021_02_12t18_00_09_35z_image_lev1.fits']

Again as we had no failed file downloads this just returns to us the files downloaded.

## 7. Metadata queries

As well as Fido providing an interface to search for data files that can be downloaded, Fido also allows you to query metadata. Currently Fido supports metadata searching from the HEK, HELIO and JSOC. 

Similar to what we have seen so far, the search results of these clients are a UnifiedResponse object which can then be indexed and the QueryResponse table accessed like an astropy table. Lets look at an example of how we can use Fido to query the HEK.

Lets query for the active regions defined by SWPC over the past month. This can be done by using the HEK client specific attributes `a.hek.attrs` 

In [55]:
result_hek = Fido.search(a.Time("2021-02-12 18:00", "2021-02-12 23:00"), 
                         a.hek.AR, )#a.hek.FRM.Name=="NOAA SWPC Observer")

In [56]:
result_hek

gs_thumburl,comment_count,hpc_bbox,meanphotoenergydensity,frm_humanflag,hgc_coord,ar_spotarearepruncert,event_coordsys,obs_levelnum,hpc_coord,event_npixels,gs_imageurl,ar_polarity,totalenergydensityunit,frm_paramset,hrc_coord,event_starttime,ar_mtwilsoncls,event_type,meantwistalpha,intensmin,obs_meanwavel,frm_url,bound_chaincode,totalphotoenergy,noposition,ar_spotarearepr,active,intensmax,frm_versionnumber,ar_pilcurvature,area_uncert,obs_dataprepurl,hpc_geom,hgc_bbox,intensmedian,chaincodetype,obs_channelid,ar_spotareareprunit,event_clippedspatial,ar_noaaclass,SOL_standard,event_avg_rating,eventtype,intensunit,hpc_boundcc,event_mapurl,frm_contact,meanvertcurrentdensity,ar_penumbracls,intensmean,bound_ccstartc1,frm_name,area_atdiskcenter,frm_identifier,obs_observatory,event_description,ar_spotarearaw,unsignedflux,boundbox_c2ur,magfluxunit,highshearareapercent,obs_firstprocessingdate,maxmagfieldstrengthunit,sharp_noaa_ars,boundbox_c2ll,frm_institute,hrc_bbox,refs_orig,meangradienthorz,gwill,ar_mcintoshcls,event_maskurl,bound_ccstartc2,ar_spotarearawunit,gs_movieurl,event_score,event_expires,hrc_boundcc,event_probability,intensvar,frm_daterun,currentdensityunit,highsheararea,event_coordunit,hpc_y,hpc_x,search_instrument,ar_numspots,kb_archivdate,kb_archivist,intenstotal,sum_overlap_scores,unsignedvertcurrent,hgs_boundcc,intensskew,obs_includesnrt,rasterscan,obs_wavelunit,kb_archivid,search_frm_name,meaninclinationgamma,currenthelicityunit,boundbox_c1ur,ar_noaanum,area_atdiskcenteruncert,boundbox_c1ll,absnetcurrenthelicity,meancurrenthelicity,event_importance_num_ratings,highshearareaunit,ar_compactnesscls,currentunit,event_testflag,maxmagfieldstrength,event_c2error,hrc_r,hgs_y,obs_title,hgs_x,hcr_checked,frm_specificid,totalphotoenergydensity,event_title,gwillunit,ar_sumnegsignedflux,obs_instrument,ar_neutrallength,event_c1error,revision,hpc_radius,meanshearangle,event_endtime,ar_spotarearawuncert,meanenergydensityunit,event_importance,meangradienttotal,event_coord2,event_coord3,event_coord1,search_observatory,ar_sumpossignedflux,area_raw,meangradientvert,concept,ar_axislength,event_pixelunit,hgc_boundcc,log_r_value,totalphotoenergyunit,hgc_x,hrc_a,hgc_y,gs_galleryid,twistunit,ar_lengthunit,hgs_coord,ar_zurichcls,bound_ccnsteps,intenskurt,event_clippedtemporal,gradientunit,rasterscantype,search_channelid,unsignedcurrenthelicity,hgs_bbox,area_unit,savncpp,obs_lastprocessingdate,refs
str131,str1,str128,object,str5,str27,object,str12,object,str30,object,str130,object,str7,str549,str41,str19,str1,str2,object,object,float64,str55,str192,object,str5,object,str4,object,float64,object,object,str1,str442,str115,object,str29,str16,str1,str1,str1,str30,object,str1,str4,str192,str1,str43,object,str1,object,object,str9,int64,str23,str3,str1,object,object,float64,str3,object,str1,str1,str1,float64,str12,str110,str1,object,object,str1,str1,object,str1,str131,str23,str1,str269,object,object,str19,str8,object,str13,float64,float64,str3,object,str19,str8,object,str20,object,str291,object,str1,str1,str2,str73,str9,object,str7,float64,object,object,float64,object,object,str1,str3,str1,str7,str5,object,float64,float64,float64,str1,float64,str4,str24,object,str1,str1,object,str3,object,float64,str1,str19,object,str19,object,str25,object,object,float64,object,float64,str3,object,object,object,str13,object,str4,str279,object,str4,float64,float64,float64,str1,str4,str1,str28,str1,object,object,str1,str4,str1,str7,object,str120,str3,object,str19,object
,0,"POLYGON((-354.800346 -420.412956,-211.87881 -414.408102,-219.12156 -362.571372,-366.925482 -368.794914,-354.800346 -420.412956))",1469.52099609375,false,POINT(58.174398 -29.981974),None,UTC-HGS-TOPO,None,POINT(-291.369228 -391.181886),None,,None,ergs/cm,/builtin/hmi.M_Ic_noLimbDark_720s.production,POINT(0.502167458733721 143.319661451754),2021-02-12T16:00:00,,AR,0.008950769901275635,None,5e-05,http://sun.stanford.edu/~mbobra/spaceweather/sharps.htm,,1.4126228311172995e+29,false,None,true,None,1.10000002

As we can see the results from the HEK returns metadata with lots of columns, we can use the `.show()` to specify the columns we want to show. Lets just print all columns out first

In [57]:
result_hek.all_colnames

['SOL_standard',
 'absnetcurrenthelicity',
 'active',
 'ar_axislength',
 'ar_compactnesscls',
 'ar_lengthunit',
 'ar_mcintoshcls',
 'ar_mtwilsoncls',
 'ar_neutrallength',
 'ar_noaaclass',
 'ar_noaanum',
 'ar_numspots',
 'ar_penumbracls',
 'ar_pilcurvature',
 'ar_polarity',
 'ar_spotarearaw',
 'ar_spotarearawuncert',
 'ar_spotarearawunit',
 'ar_spotarearepr',
 'ar_spotarearepruncert',
 'ar_spotareareprunit',
 'ar_sumnegsignedflux',
 'ar_sumpossignedflux',
 'ar_zurichcls',
 'area_atdiskcenter',
 'area_atdiskcenteruncert',
 'area_raw',
 'area_uncert',
 'area_unit',
 'bound_ccnsteps',
 'bound_ccstartc1',
 'bound_ccstartc2',
 'bound_chaincode',
 'boundbox_c1ll',
 'boundbox_c1ur',
 'boundbox_c2ll',
 'boundbox_c2ur',
 'chaincodetype',
 'comment_count',
 'concept',
 'currentdensityunit',
 'currenthelicityunit',
 'currentunit',
 'event_avg_rating',
 'event_c1error',
 'event_c2error',
 'event_clippedspatial',
 'event_clippedtemporal',
 'event_coord1',
 'event_coord2',
 'event_coord3',
 'event_co

Now lets only print the time, NOAA active region number, active region area, number of spots, and the Hale class

In [58]:
result_hek.show("event_starttime", "ar_noaanum", "area_atdiskcenter", "ar_numspots", "ar_mtwilsoncls")

event_starttime,ar_noaanum,area_atdiskcenter,ar_numspots,ar_mtwilsoncls
str19,object,int64,object,str1
2021-02-12T16:00:00,None,272829248,None,
2021-02-12T16:03:46,None,6097959936,None,
2021-02-12T16:03:46,None,1335000064,None,
2021-02-12T20:00:00,None,258148320,None,
2021-02-12T20:03:46,None,4216059904,None,
2021-02-12T20:03:46,None,902926016,None,
2021-02-12T20:03:46,None,1564899968,None,


We can also index the results table like before, using a dictionary like indexing

In [59]:
interesting_columns=["event_starttime", "ar_noaanum", "area_atdiskcenter", "ar_numspots", "ar_mtwilsoncls"]

In [60]:
result_hek["HEK", interesting_columns]

event_starttime,ar_noaanum,area_atdiskcenter,ar_numspots,ar_mtwilsoncls
str19,object,int64,object,str1
2021-02-12T16:00:00,None,272829248,None,
2021-02-12T16:03:46,None,6097959936,None,
2021-02-12T16:03:46,None,1335000064,None,
2021-02-12T20:00:00,None,258148320,None,
2021-02-12T20:03:46,None,4216059904,None,
2021-02-12T20:03:46,None,902926016,None,
2021-02-12T20:03:46,None,1564899968,None,


#### Saving results

We can then save these metadata results to a file, for example lets write it to a csv file

In [48]:
result_hek["HEK", interesting_columns].write("active_regions_may2021.csv", format="csv")

## 7. How Fido can be extended 

Fido is designed to be extensible in such a way that new data and metadata sources can be easily added and supports, providing a similar unified workflow to retrieve new data sources. This can be done both inside sunpy code and outside in, for example, an affiliated package. 

So if there is a new datasource that you would like to see in sunpy core let us know! Or even better if you are building an instrument package why now use the Fido functionality to search and retrieve your data? Some examples of affiliated packages that have recently done this is `sunpy-soar` which provides interface to the Solar Orbiter archive (SOAR), and other sunpy affiliated packages such as `radiospectra`.

To learn more about how to extend Fido please find more information here https://docs.sunpy.org/en/stable/dev_guide/contents/extending_fido.html


(COMMENT: add sunpy-soar queries here)